# Introdução

## Importação das bibliotecas necessarias

In [1]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys

## Funções auxiliares

In [2]:
def return_sql_query(query_string):
    ret = {
        "token": {
            "token": token
        },
        "sql": {
            "sql": {
                "query":query_string
            }
        }
    }
    return ret

def json_to_df(resp):
    columns = [col['name'] for col in resp.json()['columns']]
    values = resp.json()['rows']  
    return pd.DataFrame(values, columns=columns)

def rows_and_columns_to_df(columns, rows):
    columns_array = [col['name'] for col in columns]
    return pd.DataFrame(rows, columns=columns_array)

def fetch_all_data_from_response(res, initial_records_rows):
    while 'cursor' in res.json():
        res_next = {
            "token": {
                "token": token
            },
            "sql": {
                "sql": {
                    "cursor": res.json()['cursor']
                }
            }
        }
        res = requests.post(os.path.join(url_base,'sql_query'), json=res_next)
        initial_records_rows+=res.json()['rows']
        sys.stdout.write('.')
        sys.stdout.flush()
    print('(y)')
    return initial_records_rows


## Dados de acesso a api

In [3]:
from IPython.display import clear_output
import getpass

token = getpass.getpass('Digite o seu token para acesso a API:')

clear_output()

In [4]:
url_base = "https://bigdata-api.fiocruz.br"

## Bases de dados disponiveis na API atualizadas

In [5]:
params = {
          "token": token
        }
resp = requests.post(url_base+ '/show_tables', json=params)
print(resp.json())

{'databases': ['datasus-sim', 'datasus-sinasc', 'datasus-sih']}


## Pegar dados do Tabnet

In [4]:
import requests

def fetch_from_tabnet(year):
    url = 'http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sim/cnv/mat10br.def'
    # "Incremento":"%D3bitos_maternos"
    params = { "Linha":"Munic%EDpio", "Coluna":"%D3bito_investigado", "Incremento":"%D3bitos_mulheres_idade_f%E9rtil", "Arquivos":"matbr19.dbf", "pesqmes1":"Digite o texto e ache f%E1cil", "SMunic%EDpio":"TODAS_AS_CATEGORIAS__", "pesqmes2":"Digite o texto e ache f%E1cil", "SCapital":"TODAS_AS_CATEGORIAS__", "pesqmes3":"Digite o texto e ache f%E1cil", "SRegi%E3o_de_Sa%FAde_%28CIR%29":"TODAS_AS_CATEGORIAS__", "pesqmes4":"Digite o texto e ache f%E1cil", "SMacrorregi%E3o_de_Sa%FAde":"TODAS_AS_CATEGORIAS__", "pesqmes5":"Digite o texto e ache f%E1cil", "SMicrorregi%E3o_IBGE":"TODAS_AS_CATEGORIAS__", "pesqmes6":"Digite o texto e ache f%E1cil", "SRegi%E3o_Metropolitana_-_RIDE":"TODAS_AS_CATEGORIAS__", "pesqmes7":"Digite o texto e ache f%E1cil", "STerrit%F3rio_da_Cidadania":"TODAS_AS_CATEGORIAS__", "pesqmes8":"Digite o texto e ache f%E1cil", "SMesorregi%E3o_PNDR":"TODAS_AS_CATEGORIAS__", "SAmaz%F4nia_Legal":"TODAS_AS_CATEGORIAS__", "SSemi%E1rido":"TODAS_AS_CATEGORIAS__", "SFaixa_de_Fronteira":"TODAS_AS_CATEGORIAS__", "SZona_de_Fronteira":"TODAS_AS_CATEGORIAS__", "SMunic%EDpio_de_extrema_pobreza":"TODAS_AS_CATEGORIAS__", "pesqmes14":"Digite o texto e ache f%E1cil", "SCap%EDtulo_CID-10":"TODAS_AS_CATEGORIAS__", "pesqmes15":"Digite o texto e ache f%E1cil", "SGrupo_CID-10":"TODAS_AS_CATEGORIAS__", "pesqmes16":"Digite o texto e ache f%E1cil", "SCategoria_CID-10":"TODAS_AS_CATEGORIAS__", "STipo_causa_obst%E9tr":"TODAS_AS_CATEGORIAS__", "pesqmes18":"Digite o texto e ache f%E1cil", "SFaixa_Et%E1ria":"TODAS_AS_CATEGORIAS__", "pesqmes19":"Digite o texto e ache f%E1cil", "SFaixa_Et%E1ria_OPS":"TODAS_AS_CATEGORIAS__", "pesqmes20":"Digite o texto e ache f%E1cil", "SFaixa_Et%E1ria_det":"TODAS_AS_CATEGORIAS__", "SCor%2Fra%E7a":"TODAS_AS_CATEGORIAS__", "SEscolaridade":"TODAS_AS_CATEGORIAS__", "SEstado_civil":"TODAS_AS_CATEGORIAS__", "SLocal_ocorr%EAncia":"TODAS_AS_CATEGORIAS__", "SMorte_grav/puerp":"TODAS_AS_CATEGORIAS__", "S%D3bito_investigado":"TODAS_AS_CATEGORIAS__", "formato":"prn", "mostre":"Mostra"}
    params['Arquivos'] = 'matbr'+year+'.dbf'
    print('!')
    resp = requests.post(url, data=params)
    print('@')
    print(resp.status_code)
    return resp.text

print(fetch_from_tabnet('12'))

!


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

# Proporção de óbitos de mulher em idade fértil investigados (SIM)

A ideia inicial era explorar os dados de todo o Brasil do período de 2012 a 2019.
Como a tabela SIH-RD tratada e filtrada que temos é do período de 2018 até 2019 e apenas do Estado de São Paulo, iremos utilizar o mesmo subconjunto.

In [8]:
%time
sql_total_obitos = return_sql_query(
    "SELECT CODMUNRES, ano_obito as Ano, count(*) as Qtd_Total  \
    FROM \"datasus-sim\" \
    WHERE ano_obito>=2012 AND \
    ano_obito<=2019 AND \
    idade_obito_anos>=10 AND \
    idade_obito_anos<=49 AND \
    SEXO=2 \
    GROUP BY CODMUNRES, ano_obito")
resp_sql_total_obitos = requests.post(os.path.join(url_base,'sql_query'), json=sql_total_obitos)
record_columns=resp_sql_total_obitos.json()['columns']
record_rows=resp_sql_total_obitos.json()['rows']

fetch_all_data_from_response(resp_sql_total_obitos, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_total_obitos = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.58 µs
........................................(y)
Record rows length : 39122


In [9]:
%time   
sql_obitos_investigados = return_sql_query(
    "SELECT CODMUNRES, ano_obito as Ano, count(*) as Qtd_Obitos_Investigados \
    FROM \"datasus-sim\" WHERE ano_obito>=2012 AND \
    ano_obito<=2019 AND \
    idade_obito_anos>=10 AND \
    idade_obito_anos<=49 AND \
    TPPOS LIKE \'S\' AND \
    SEXO=2 \
    GROUP BY CODMUNRES, ano_obito")

resp_sql_obitos_investigados = requests.post(os.path.join(url_base,'sql_query'), json=sql_obitos_investigados)
record_columns=resp_sql_obitos_investigados.json()['columns']
record_rows=resp_sql_obitos_investigados.json()['rows']

fetch_all_data_from_response(resp_sql_obitos_investigados, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_obitos_investigados = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs
............................(y)
Record rows length : 27372


In [10]:
df_obitos_investigados.head()

,CODMUNRES,Ano,Qtd_Obitos_Investigados
0,110000,2013,1
1,110000,2015,1
2,110000,2016,3
3,110000,2019,1
4,110001,2013,1


In [11]:
df_total_obitos.head()

,CODMUNRES,Ano,Qtd_Total
0,110000,2013,1
1,110000,2014,2
2,110000,2015,1
3,110000,2016,4
4,110000,2018,1


In [12]:
df_obitos_investigados['Ano'] = df_obitos_investigados['Ano'].astype(int)
df_total_obitos['Ano'] = df_total_obitos['Ano'].astype(int)
df_merge = pd.merge(df_obitos_investigados, df_total_obitos, how='right', on=['CODMUNRES','Ano'])
df_merge = df_merge.fillna(0)
df_merge['Ratio'] = df_merge['Qtd_Obitos_Investigados']/(df_merge['Qtd_Total'])
df_merge.head()

,CODMUNRES,Ano,Qtd_Obitos_Investigados,Qtd_Total,Ratio
0,110000,2013,1.0,1,1.00
1,110000,2014,0.0,2,0.00
2,110000,2015,1.0,1,1.00
3,110000,2016,3.0,4,0.75
4,110000,2018,0.0,1,0.00


In [13]:
%time
df_merge.to_csv('./csv_files/W-24-proporcao_obitos_mulher_idade_fertil_investigados.csv',index=False)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10.3 µs
